In [1]:
import os
import fnmatch

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from lr_video_face.plots import *

from lir import Xy_to_Xn, metrics

In [2]:

home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database_21')
#db_name = 'colab_main_dataset'
# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'output_5')

#Traducción de nombres
dic_emb = {'Facenet512': 'FaceNet', 'QMagFace':'QMagFace', 'ArcFace':'ArcFace'}
dic_qua = {'tface' : 'SDD-FIQA','ser_fiq': 'SER-FIQ'}

In [3]:

files = []
for root, dir, filenames in os.walk(output_dir):
    for filename in fnmatch.filter(filenames, 'dat*.pd'):
        files.append(os.path.join(root, filename))
   

In [4]:
def generate_plot(file):

    df0 = pd.read_pickle(file)

    if len(pd.unique(df0.Detector)) == 1 and len(pd.unique(df0.Calibrator)) == 1:

        cols = list(pd.unique(df0['Embedding Model']))
        rows = list(pd.unique(df0['Quality Model']))

        #generamos la gráfica con subplots
        fig,ax = plt.subplots(nrows = len(rows), ncols = len(cols), squeeze= False, figsize= (25,10))
        
        for index, df1 in df0.iterrows():

            col = cols.index(df1['Embedding Model'])
            row = rows.index(df1['Quality Model'])

            ax1 = ax[row][col]
            #ax1.ylabel = df1['Embedding Model']
            #ax1.set_title ( f"Quality Model: {dic_qua [df1['Quality Model']]}")
            ax1.set_title ( f"Embedding Model: {dic_emb [df1['Embedding Model']]}",fontsize = 14)

            subplot_new(ax1,df1.Results, df1.Cllr)
            #ax1.set(ylabel= f"Embedding Model: {dic_emb[df1['Embedding Model']]}\n Cllr")
            
            #ax1.set(ylabel= f"Quality Model: {dic_qua[df1['Quality Model']]}\n Cllr")
            ax1.set_ylabel(f"Quality Model: {dic_qua[df1['Quality Model']]}\n Cllr",fontsize = 14)
            ax1.set_xlabel('% of discarded pairs',fontsize=14)
        plt.suptitle('$C_{llr}$ Values',fontsize=18)

        #leyenda global
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14, borderaxespad=0.)
        fig.tight_layout()
        #savefig = os.path.join(self.experiments.output_dir, f"cllr_summary_ESX{self.experiments.embedding_model_as_scorer}")
        savefig = os.path.join(os.path.dirname(file), os.path.splitext(file)[0]+'_v4')
        plt.savefig(savefig, dpi=600)
        plt.close() 

In [5]:
def generate_latex(file):
    df0 = pd.read_pickle(file)


    
    

    if len(pd.unique(df0.Detector)) == 1 and len(pd.unique(df0.Calibrator)) == 1:
        df_latex = pd.DataFrame()

        for index, df1 in df0.iterrows():    

            emb_model = df1['Embedding Model']
            qua_model = df1['Quality Model']
            # the results are only received for 2015
            lrs_predicted = df1.Results["lrs_predicted"]
            y_test = df1.Results["y_test"]
            dropouts = df1.Results["quality_drops"]
            common_attributes = df1.Results['common_attributes']
    
            cllr_participants = np.mean(df1.Cllr)

            # imagen promedio (17 comparisons)
            lr_avg = df1.Results["lrs_predicted_2015"]
            y_avg = df1.Results["y_test_2015"]    
    
            cllr_avg = metrics.cllr(np.asarray(lr_avg), np.asarray(y_avg)) 
            df_latex = df_latex.append({'Embedding_model': 'Participants',
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_participants},ignore_index = True)
            df_latex = df_latex.append({'Embedding_model': 'Average Image',
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_avg},ignore_index = True)

            #get cllr per dropout

            
            for d in set(dropouts):
                lr_d = [lr for lr,dropout in zip(lrs_predicted,dropouts) if dropout== d ]
                y_d = [y for y,dropout in zip(y_test,dropouts) if dropout== d ]
                cllr_d = metrics.cllr(np.asarray(lr_d), np.asarray(y_d))

            #solo cambio en el momento de plotear
                df_latex = df_latex.append({'Embedding_model': dic_emb[emb_model],
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_d,
                                             'Dropout': 100*(1-d)},ignore_index = True)

            df_latex.sort_values(by= 'Dropout', inplace = True)
            #df_latex.dropna(inplace= True)

            

            # get cllr per common attributes
            # filter results when dropout = 1 
            lrs = [lr for lr,drop in zip(lrs_predicted,dropouts) if drop== 1] 
            ys = [y for y,drop in zip(y_test,dropouts) if drop== 1]
            comatt = [ca for ca,drop in zip(common_attributes,dropouts) if drop== 1]


            #df_plot2 = pd.DataFrame()
            for c in set(comatt):
                lr_c = [lr for lr,com in zip(lrs,comatt) if com== c ]
                y_c = [y for y,comm in zip(ys,comatt) if comm== c ]
                cllr_c = metrics.cllr(np.asarray(lr_c), np.asarray(y_c))
                df_latex = df_latex.append({'Embedding_model': dic_emb[emb_model],
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_c,
                                             'Common Attributes': c},ignore_index = True)
        
            df_latex.sort_values(by= 'Common Attributes', inplace = True)
            #df_latex.dropna(inplace= True)   

#----
        latex_table = df_latex.to_latex(index=False)
        savelatex = os.path.join(os.path.dirname(file), os.path.splitext(file)[0]+'.tex')

        with open (savelatex,'w') as f:
            f.write(latex_table)

In [6]:
for file in files:
    generate_plot(file)
    generate_latex(file)


0.8369419513906209 [0.6834495  0.8029329  0.96072678]
0.9168676702366412 [0.73571939 0.90494434 1.09722767]
0.878205128627314 [0.70845145 0.85715813 1.03523177]
0.971197312989641 [0.76762712 0.97905594 1.22745708]
0.9447225394802354 [0.75374949 0.94435726 1.16292147]
0.9995283817131855 [0.78624896 1.01717962 1.29552394]
1.0291906448779278 [0.80217662 1.05421222 1.37090811]
1.050763789392876 [0.81628618 1.08866719 1.44192645]
1.0653906319115354 [0.82901316 1.12446447 1.49035047]
1.0777992243829457 [0.83941943 1.15804076 1.5374042 ]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram

1.0789619583860222 [0.23221742 0.4292182  0.99998837]
1.1365623477750217 [0.28089591 0.59877001 1.15760097]
1.1213500533736724 [0.2497919  0.47579684 1.0736271 ]
1.2073100667569907 [0.34229947 0.76345689 1.32722507]
1.1907361201425157 [0.30947539 0.67846101 1.26221827]
1.203722409241233 [0.37910388 0.82715428 1.36304148]
1.1987210852515953 [0.4059374  0.85491324 1.37973498]
1.211934187668891 [0.43886075 0.88090503 1.38735242]
1.2167074446215345 [0.4690882  0.90954696 1.4063193 ]
1.2170929977449156 [0.48656419 0.92833669 1.41375169]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram

0.31512489798788335 [0.12636053 0.2303199  0.39284393]
0.4233671013294673 [0.19238844 0.33997512 0.60355773]
0.37083701754120424 [0.16246037 0.28877406 0.48425296]
0.578954539574634 [0.24123376 0.44624028 0.9340788 ]
0.48925868917491866 [0.2175599  0.38570057 0.74120806]
0.6798717791312073 [0.26383811 0.53007473 1.20461767]
0.7567944648495285 [0.28758578 0.61643483 1.43472441]
0.8336682865340725 [0.30560999 0.71292534 1.65526482]
0.8927821625634091 [0.31977639 0.80785332 1.83354105]
0.9464823334042575 [0.32817092 0.90667689 1.99847233]


...

/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram

1.2727477673520111 [1.93283838 2.18591691 2.34707934]
1.307413933775262 [1.93735292 2.28757703 2.4219123 ]
1.2937223053045552 [1.94499585 2.23855541 2.39268841]
1.3269741461538789 [1.95410879 2.35671333 2.46278834]
1.3181001443009601 [1.94169803 2.33279968 2.45035614]
1.3381423658194742 [1.97655069 2.37667776 2.47106137]
1.3511420378484162 [1.9946077  2.38815331 2.47573852]
1.3581665376330614 [1.99844459 2.40023248 2.48013004]
1.3621213112266417 [2.00738053 2.41050175 2.483599  ]
1.3654172195080494 [2.00994482 2.41813619 2.4870718 ]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram

1.014872245124008 [0.94608171 0.9876889  1.03375663]
1.0123581609986885 [0.935169   0.96768639 1.03367166]
1.01311049241197 [0.93941489 0.97658679 1.03377709]
1.0066128619475105 [0.93107356 0.95461389 1.02908807]
1.0072121497480198 [0.932552   0.9598274  1.02967351]
1.0061826446804158 [0.93024936 0.95099794 1.02825767]
1.0060897756616762 [0.92993371 0.94900273 1.02785129]
1.0017928779041387 [0.92896282 0.9464237  1.02347085]
0.9994337347376678 [0.92846511 0.94507877 1.01815381]
0.9990827413158518 [0.92814603 0.94406573 1.01616322]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram